In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from catalyst import run_algorithm
from catalyst.api import (record,order_target_percent,order,symbol)

def is_profitabel(sell_price,buy_price,sell_market,buy_market):
    sell_fee=get_market_fee(sell_market,sell_price)
    buy_fee=get_market_fee(buy_market,buy_price)
    expected_profit=sell_price-buy_price-sell_fee-buy_fee
    if expected_profit>0:
        return True
    else:
        return False

def get_market_fee(market,price):
    return round(market.api.fees['trading']['taker']*price,5)


def adjusted_price(price,slippage):
    adj_sell_price=price*(1-slippage)
    adj_buy_price=price*(1+slippage)
    return adj_sell_price,adj_buy_price



def initialize(context):
    context.bitfinex=context.exchanges['bitfinex']
    context.poloniex=context.exchanges['poloniex']
    #
    context.bitfinex_trading_pair=symbol('btc_usdt',context.bitfinex.name)
    context.poloniex_trading_pair=symbol('btc_usdt',context.poloniex.name)
    
def handle_data(context,data):
    bitfinex_price=data.current(context.bitfinex_trading_pair,'price')
    poloniex_price=data.current(context.poloniex_trading_pair,'price')
    
    orders = context.blotter.open_orders
    if len(orders) > 0:
        return
    
    #USUALY SLIPAGE IS USED TO BE AROUND 3%
    slippage=0.03
    
    sell_b,buy_b=adjusted_price(bitfinex_price,slippage)
    sell_p,buy_p=adjusted_price(poloniex_price,slippage)
    
    
    #record()
    if sell_b>buy_p:
        print('buy on poloniex at {} and sell on bitfinex at {} '.format(buy_p,sell_b))
        print()
        order(asset=context.poloniex_trading_pair,
             amount=1,
             limit_price= buy_p)
        order(asset=context.bitfinex_trading_pair,
             amount=-1,
             limit_price=sell_b)
    
    elif buy_p<sell_p:
        print('buy on bitfinex and sell on poloniex ')
        order(asset=context.poloniex_trading_pair,
             amount=-1,
             limit_price=sell_p)
        order(asset=context.bitfinex_trading_pair,
             amount=1,
             limit_price=buy_b)

    
    
#IF WANT TO RECORD SOMETHING AND PLOT AFTER THE RUN PUT IT IN THE ANALYZE FUNCTION      
def analyze(context,perf):
    pass
    
    
if __name__ == '__main__':
    run_algorithm(
            capital_base=1000,
            data_frequency='minute',
            initialize=initialize,
            handle_data=handle_data,
            live =False,
            analyze=analyze,
            exchange_name='bitfinex,poloniex',
            #algo_namespace=NAMESPACE,
            quote_currency='usdt',
            start=pd.to_datetime('2017-12-12', utc=True),
            end=pd.to_datetime('2017-12-14', utc=True),
        )

                
   





In [ ]:
   
#TO PLACE AN ORDER WITHOUT CONSIDERING THE TRADING FEE AND SLIPAGE

if bitfinex_price>poloniex_price:
        print('buy on poloniex and sell on bitfinex ')
        order(asset=context.poloniex_trading_pair,
             amount=1,
             limit_price= poloniex_price)
        order(asset=context.bitfinex_trading_pair,
             amount=-1,
             limit_price=bitfinex_price)
    elif bitfinex_price<poloniex_price:
        print('buy on bitfinex and sell on poloniex ')
        order(asset=context.poloniex_trading_pair,
             amount=-1,
             limit_price= poloniex_price)
        order(asset=context.bitfinex_trading_pair,
             amount=1,
             limit_price=bitfinex_price)
   
    


    